In [31]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [32]:
batch_size=32
buffer_size=1000

In [33]:
(x_train, y_train), (x_test,y_test) = tf.keras.datasets.cifar10.load_data()
objects = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']


In [34]:
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

In [35]:
def preprocess_cifar10(img, label):
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, (75, 75))
    return tf.keras.applications.inception_v3.preprocess_input(img) , label
dataset_train_processed = dataset_train.shuffle(buffer_size).batch(batch_size).map(preprocess_cifar10)
dataset_test_processed = dataset_test.batch(batch_size).map(preprocess_cifar10)


In [36]:
inception_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(75,75,3)
)


In [41]:

x = inception_model.output
x= keras.layers.GlobalMaxPooling2D()(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(1024, activation="relu")(x)
x = keras.layers.Dense(128, activation="relu")(x)
output = keras.layers.Dense(10, activation="softmax")(x)
model=keras.Model(inputs=inception_model.input, outputs = output)

In [42]:
for inception_layer in inception_model.layers:
    inception_layer.trainable= False

In [43]:

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [44]:

model.fit(x=dataset_train_processed, validation_data = dataset_test_processed, epochs=3)

Epoch 1/3
1563/1563 [==============================] - 324s 205ms/step - loss: 1.2455 - accuracy: 0.5609 - val_loss: 1.0609 - val_accuracy: 0.6254
Epoch 2/3
1563/1563 [==============================] - 327s 209ms/step - loss: 1.0418 - accuracy: 0.6320 - val_loss: 1.0140 - val_accuracy: 0.6478
Epoch 3/3
1563/1563 [==============================] - 339s 217ms/step - loss: 0.9409 - accuracy: 0.6666 - val_loss: 1.0180 - val_accuracy: 0.6402
